In [ ]:
%pip install pinecone

In [78]:
import requests
from bs4 import BeautifulSoup
import os
import ollama
import json
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from pinecone import Pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import json
import time
import pandas as pd

HOME_URL = "https://hibizsolutions.com/"
SERVICES_URL = "https://hibizsolutions.com/salesforce-services/"
pc = Pinecone(api_key="pcsk_7HNY1h_9DrYuFREEMihSepXtfj7VmwMCL8JHf2H8e4hkGTtXtysJn2efm6kDK8axttVZDR")

In [79]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}
response = requests.get(HOME_URL, headers=headers)

service_response = requests.get(SERVICES_URL, headers=headers)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')
service_soup = BeautifulSoup(service_response.text, 'html.parser')

In [14]:
testimonials = soup.find_all('div', class_ = 'qodef-e-inner')

testimonial_list = []

#print(testimonials)
# Extract testimonial text and author (if available)
for t in testimonials:
    print("------------")
    quote_content = t.find('p', class_ = 'qodef-e-text')
    author = t.find('h5', class_ = 'qodef-e-author-name')
    author_job = t.find('span', class_ = 'qodef-e-author-job').get_text(strip=True)
    
    author_job_role = author_job.split(' - ')[0] if ' - ' in author_job else 'Unknown'
    author_job_company = author_job.split(' - ')[1] if ' - ' in author_job else 'Unknown'

    testimonial_data = {
        "author_name": author.get_text(strip=True) if author else 'Unknown',
        "author_job_role": author_job_role,
        "author_company": author_job_company,
        "testimonial": quote_content.get_text(strip=True) if quote_content else 'No quote found'
    }

    testimonial_list.append(testimonial_data)

# Print JSON list
print(json.dumps(testimonial_list, indent=2))
    

------------
------------
------------
------------
------------
------------
------------
[
  {
    "author_name": "Mike Klas",
    "author_job_role": "Director of Expert Services",
    "author_company": "Talkdesk, Inc. USA",
    "testimonial": "Hibiz has been an invaluable partner throughout my professional journey across Navis, Cargotec, Rootstock, and Talkdesk. From spearheading custom development projects to providing consistent support, their reliability, speed, and competitive rates have remained unmatched. What sets them apart is their genuine commitment to my success, making them a trusted ally rather than just a service provider. With Hibiz, excellence isn't just a promise\u2014it's a guarantee."
  },
  {
    "author_name": "Casie Archuleta",
    "author_job_role": "Director, Global Professional Service",
    "author_company": "Talkdesk, Inc. USA",
    "testimonial": "Working with Raj and the exceptional team at Hibiz over the past four years has been nothing short of extraor

In [ ]:
partners = soup.find_all('div', class_='elementor-widget-container')

print("length :", len(partners))
for partner in partners:
    print("\n-----------------------")
    print(partner)

length : 74

-----------------------
<div class="elementor-widget-container">
<div class="wp-block-themepunch-revslider">
<!-- START Main Home REVOLUTION SLIDER 6.6.14 --><p class="rs-p-wp-fix"></p>
<rs-module-wrap data-source="gallery" id="rev_slider_8_1_wrapper" style="visibility:hidden;background:transparent;padding:0;">
<rs-module data-version="6.6.14" id="rev_slider_8_1" style="">
<rs-slides style="overflow: hidden; position: absolute;">
<rs-slide data-in="o:0;" data-key="rs-10" data-out="a:false;" data-thumb="//143.198.70.96/wp-content/uploads/2021/11/Rev-slider-img-1.jpg" data-title="Slide" style="position: absolute;">
<img alt="a" class="rev-slidebg tp-rs-img rs-lazyload" data-lazyload="//143.198.70.96/wp-content/uploads/2021/11/Rev-slider-img-1.jpg" data-no-retina="" decoding="async" fetchpriority="high" height="1100" src="//hibizsolutions.com/wp-content/plugins/revslider/public/assets/assets/dummy.png" title="Rev-slider-img-1" width="1920"/>
<!--
--><rs-layer data-basealign="

## Extract Partners

In [ ]:
output_dir = 'downloaded_images'
os.makedirs(output_dir, exist_ok=True)

partners = service_soup.find_all('div', class_='gallery galleryid-10466 gallery-columns-2 gallery-size-full')[-1]
# Find all dt tags inside dl.gallery-item
dt_tags = partners.select('dl.gallery-item dt.gallery-icon')
print("dt_tags:", dt_tags)
print(len(dt_tags))


for idx, dt in enumerate(dt_tags, 1):
    img = dt.find('img')
    print("img:", img)
    if img and img.get('src'):
        img_url = img['src']
        file_name = os.path.join(output_dir, f'image_{idx}{os.path.splitext(img_url)[1]}')
        print(f'Downloading: {img_url} -> {file_name}')
        try:
            response = requests.get(img_url)
            if response.status_code == 200:
                with open(file_name, 'wb') as f:
                    f.write(response.content)
            else:
                print(f'Failed to download {img_url}: Status code {response.status_code}')
        except Exception as e:
            print(f'Error downloading {img_url}: {e}')


In [10]:
partner_names = []
for i in range(1,9):

  with open(f'downloaded_images/image_{i}.png', 'rb') as file:
    response = ollama.chat(
      model='gemma3:4b',
      messages=[
        {
          'role': 'user',
          'content': 'Extract the company/application name from the image.',
          'images': [file.read()],
        },
      ],
    )

  partner_names.append(response['message']['content'])

print("Extracted Partner Names:", partner_names)

Extracted Partner Names: ['The company/application name is **Databricks**.', 'Power BI', 'The company/application name is **Rootstock**.', 'The company/application name in the image is **Microsoft**.', 'The company/application name in the image is **Azure**.', 'The company/application name from the image is **Microsoft Azure**.', 'The company/application name in the image is **AWS** (Amazon Web Services).', 'The company name from the image is **Salesforce**.']


In [ ]:
services = soup.find_all('div', class_='e-con-inner')
i = 0
print("length :", len(services))
for service in services:
    print("\n-----------------------", i)
    i += 1
    print(service)

In [ ]:
for each_div in services:
    print(each_div.prettify())

    service_headings = each_div.find_all("h2")
    print("\n-----------------------")
    print(service_headings)
    # print("service_headings:", service_headings)
    # for each in service_headings:
    #     print("\n-----------------------")
    #     print("Service Heading:", each.get_text(strip=True))

In [ ]:

def get_services_provided():
    services_content = soup.find_all('h2', class_ = "elementor-heading-title elementor-size-default")
    print(len(services_content))

    services_provided = []
    for tag in services_content:
        services_provided.append(tag.get_text(strip=True))

    print(services_provided)   

9
['Salesforce', 'ERP Services', 'AI & Data Analytics', 'Digital Transformation', 'Consulting', 'Software Product Engineering', 'Software Integrations', 'Product Helpdesk', 'Custom Apps Development']
